In [1]:
import numpy as np
from collections import defaultdict

In [2]:
D = np.array([
    [0, 0, 1, 0, 1],
    [0, 1, 1, 1, 1],
    [0, 2, 0, 1, 0],
    [0, 2, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1],
    [1, 2, 0, 0, 1],
    [1, 1, 0, 1, 1],
    [2, 0, 1, 0, 1],
    [2, 1, 1, 0, 1],
    [2, 1, 0, 1, 1],
    [2, 0, 1, 1, 0],
    [2, 1, 0, 1, 0]
])

In [9]:
D[D[:, 1] == 2]

array([[0, 2, 0, 1, 0],
       [0, 2, 0, 1, 0],
       [1, 2, 0, 0, 1]])

In [10]:
def compute_entropy(X, target):
    counts = defaultdict(float)
    N = 0.0
    for x in X:
        counts[x[target]] += 1
        N += 1
    entropy = 0.0
    for k, v in counts.items():
        p = (v/N)
        entropy +=  p * np.log2(p)
    return -entropy


def compute_conditional_entropy(X, target, conditioning_feature):
    counts = defaultdict(float)
    subsets = defaultdict(list)
    N = 0.0
    for x in X:
        counts[x[conditioning_feature]] += 1.0
        subsets[x[conditioning_feature]].append(x)
        N += 1.0 
    conditional_entropy = 0.0
    for k, v in subsets.items():
        factor = counts[k] / N 
        conditional_entropy += factor * compute_entropy(v, target)
    return conditional_entropy
     
        
def compute_information_gain(X, target, feature):
    entropy = compute_entropy(X, target)
    conditional_entropy = compute_conditional_entropy(X, target, feature) 
    return entropy - conditional_entropy
    
    

In [11]:
compute_entropy(D, 4)

0.9402859586706311

In [12]:
compute_conditional_entropy(D, 4, 0)

0.6935361388961918

In [14]:
compute_information_gain(D, 4, 0)

0.24674981977443933

In [16]:
class TreeNode:
    def __init__(self, feature, probs={}):
        self.feature = feature
        self.branches = {} 
        self.probs = probs
    
    def make_decision(self, data):
        branch = self.branches.get(data[self.feature], None)
        if branch is None:
            pair = max(self.probs.items(), key=lambda x: x[1])
            return pair[0]
        return branch.make_decision(data)

In [17]:
def get_best_splitting_feature(X, target, used_features):
    num_features = len(X[0])
    best_gain = -1
    best_feature = -1
    for i in range(num_features):
        if i != target and i not in used_features:
            current_gain = compute_information_gain(D, target, i)
            if current_gain > best_gain:
                current_gain = best_gain
                best_feature = i 
    if best_feature != -1:
        return best_feature 
    return -1

In [18]:
def get_probs(X, target):
    N = len(X)
    counts = defaultdict(float)
    for x in X:
        counts[x[target]] += 1.0
    counts = dict(counts)
    for k, v in counts.items():
        counts[k] = v / N 
    return counts

In [19]:
def build_decision_tree(X, target, used_features=set()):
    if len(used_features) == (len(X[0]) - 1):
        return None
    feature = get_best_splitting_feature(X, target, used_features)
    probs = get_probs(X, target)
    used_features = used_features | set([feature])
    root = TreeNode(feature, probs)
    possible_values = set(X[:, feature])
    for possible_value in possible_values:
        X1 = X[X[:, feature] == possible_value]
        root.branches[possible_value] = build_decision_tree(X1, target, used_features)
    return root 

In [20]:
root = build_decision_tree(D, 4)

In [21]:
root.make_decision(D[1])

1

In [15]:
get_probs(D, 4)

{1: 0.6428571428571429, 0: 0.35714285714285715}